In [8]:
# Which Actors appear most frequently in high-rated movies (e.g., IMDb ratings > 8)

# Import
import requests
import json
import matplotlib as mp
import pandas as pd
from pprint import pprint
from config import tmdb_api_key
from config import omdb_api_key

# TMDB Setup 

# t_api_key = tmdb_api_key
# t_base_url = "https://api.themoviedb.org/3/discover/movie"


In [21]:
# Import Credits and setting id as index

credits_df = pd.read_csv("Resources/credits.csv")
credits_df.set_index("id", inplace=True)

# Import Movies Metadata and setting id as index

movies_df = pd.read_csv("Resources/movies_metadata.csv")
movies_df.set_index("id", inplace=True)

In [ ]:
# Dropping unneeded columns for movie dataframe
clean_movies_df = movies_df.drop(columns=["adult", 
                                          "belongs_to_collection", 
                                          "budget", 
                                          "genres", 
                                          "homepage", 
                                          "original_language", 
                                          "overview",
                                          "popularity",
                                          "poster_path",
                                          "production_companies",
                                          "production_countries",
                                          "revenue", 
                                          "runtime", 
                                          "spoken_languages", 
                                          "status", 
                                          "tagline", 
                                          "video"])
clean_movies_df

In [ ]:
clean_credits_df = credits_df.drop(columns = ["crew"])
clean_credits_df.head()

In [ ]:
for actor in clean_credits_df[0:4]:
    print(actor)

In [ ]:
for i, actor in list(clean_credits_df.iterrows())[0:5]:
    print(actor)

In [ ]:
print(dir(clean_credits_df))

In [ ]:
pd.set_option('display.max_colwidth', None)
print(credits_df.iloc[[862]])

In [ ]:
actors = {}
for _, movie in clean_credits_df.iterrows():
  for actor in movie['cast'][0:5]:
    name = actor.get('name')
    if name not in actors.keys():
      actors[name] = 0
    actors[name] = actors[name] + 1
print(actors)